<a href="https://colab.research.google.com/github/ArtemNechaev/stepik_nnets/blob/main/task8_translate_de2en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translate German to English


In [ ]:
!git clone https://github.com/ArtemNechaev/stepik_nnets & pip install -r requirements.txt
#!pip install -U spacy 
#!python -m spacy download en_core_web_sm
#!python -m spacy download de_core_news_sm 
#!wget 'http://vectors.nlpl.eu/repository/20/45.zip'

fatal: destination path 'stepik_nnets' already exists and is not an empty directory.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --v (--verbose, --version?)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


from torchtext.datasets import Multi30k
from collections import OrderedDict, Counter

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, FastText, vocab
from torchtext.datasets import Multi30k
from typing import Iterable, List
#from torchtext.data import Field, BucketIterator

from stepik_nnets.sec2sec.data import TranslationDataset, sequential_transforms, TensorTransform, myFastText
from stepik_nnets.sec2sec.models.base_model import Encoder, Decoder, Seq2Seq, Attention, MyAttention 
from stepik_nnets.sec2sec.models.only_gru import OnlyGRU 

from stepik_nnets.sec2sec.engine import train, evaluate, predict_with_model


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy

import random
import math
import time
from tqdm import tqdm
import pickle

import zipfile
#import gensim


ModuleNotFoundError: ignored

In [ ]:
SEED = 0
pretrained_embed = True

ENC_EMB_DIM = 300
DEC_EMB_DIM = 100

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=1):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

In [ ]:

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

ln_pair = (SRC_LANGUAGE, TGT_LANGUAGE)

# Place-holders
token_transform = {}
vocab_transform = {}


# Create source and target language tokenizer. Make sure to install the dependencies.

token_transform[TGT_LANGUAGE] = tokenize_text_simple_regex#get_tokenizer('spacy', language='en_core_web_sm')
token_transform[SRC_LANGUAGE] = tokenize_text_simple_regex#get_tokenizer('spacy', language='de_core_news_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]].lower())

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

train_iter = ((token_transform[SRC_LANGUAGE](t[0]) , token_transform[TGT_LANGUAGE](t[1]))
    for t in Multi30k(split='train', language_pair=ln_pair))

train_iter_src, train_iter_trg = zip(*train_iter)


if pretrained_embed:
  ft_embed = myFastText(language='de')
  vocab_transform[SRC_LANGUAGE] = vocab(ft_embed.stoi,
                                                    min_freq=0,
                                                    specials=special_symbols,
                                                    special_first=True)
else:
  vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(train_iter_src,
                                                      min_freq=1,
                                                      specials=special_symbols,
                                                      special_first=True)

vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(train_iter_trg,
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)



#for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    #train_iter = Multi30k(split='train', language_pair=ln_pair)
    # Create torchtext's Vocab object
   # vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    #min_freq=1,
                                                    #specials=special_symbols,
                                                    #special_first=True)

    # Set UNK_IDX as the default index. This index is returned when the token is not found.
    # If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.

vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)


# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in ln_pair:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               TensorTransform(BOS_IDX, EOS_IDX)) # Add BOS/EOS and create tensor

if pretrained_embed:
  src_embed = nn.Embedding(len(vocab_transform[SRC_LANGUAGE]), 300, padding_idx=PAD_IDX)
  src_embed.load_state_dict({'weight': torch.cat([src_embed.weight[:4].data, ft_embed.vectors])})
  src_embed.requires_grad_(False)
else:
  src_embed = None

/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:181: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


In [ ]:
u,s,vt = torch.linalg.svd(ft_embed.vectors)

In [ ]:
train_iter = Multi30k(split='train', language_pair=ln_pair)
train_dataset = TranslationDataset( train_iter, text_transform, PAD_IDX, ln_pair = ln_pair)
train_iterator = DataLoader(train_dataset, batch_size=30, shuffle =True, collate_fn = train_dataset.pad_collate_fn)

valid_iter = Multi30k(split='valid', language_pair=ln_pair)
val_dataset = TranslationDataset( valid_iter, text_transform, PAD_IDX, ln_pair = ln_pair)
valid_iterator = DataLoader(val_dataset, batch_size=30, collate_fn = train_dataset.pad_collate_fn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:181: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda')

In [ ]:
INPUT_DIM = len(vocab_transform[SRC_LANGUAGE])
OUTPUT_DIM = len(vocab_transform[TGT_LANGUAGE])

ENC_HID_DIM = 100
DEC_HID_DIM = 100
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT, embedding=src_embed)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn, embedding=None)

model = Seq2Seq(enc, dec, PAD_IDX, BOS_IDX, EOS_IDX, device).to(device)
#model = OnlyGRU(INPUT_DIM, OUTPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, PAD_IDX, EOS_IDX, device, embedding=src_embed).to(device)

In [ ]:
if not pretrained_embed:
  def init_weights(m): 
    for name, param in m.named_parameters(): 
      if 'weight' in name: 
        nn.init.normal_(param.data, mean=0, std=0.01) 
      else: nn.init.constant_(param.data, 0)

  model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Модель содержит {count_parameters(model):,} параметров')

Модель содержит 5,365,066 параметров


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
sched = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
CLIP = float('inf')
best_valid_loss = float('inf')
model.forward_mode = 'next_word'

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP, device)
    valid_loss = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    sched.step(valid_loss)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'translate_model.pt')
    
    print(f'Эпоха: {epoch+1:02} | Время: {epoch_mins}m {epoch_secs}s')
    print(f'Перплексия (обучение): {math.exp(train_loss):7.3f}')
    print(f'Перплексия (валидация): {math.exp(valid_loss):7.3f}')

Эпоха: 01 | Время: 0m 57s
Перплексия (обучение):  16.208
Перплексия (валидация):  53.192
Эпоха: 02 | Время: 0m 58s
Перплексия (обучение):  15.512
Перплексия (валидация):  52.584
Эпоха: 03 | Время: 0m 59s
Перплексия (обучение):  14.988
Перплексия (валидация):  52.396
Эпоха: 04 | Время: 0m 57s
Перплексия (обучение):  14.436
Перплексия (валидация):  51.930
Эпоха: 05 | Время: 0m 57s
Перплексия (обучение):  14.128
Перплексия (валидация):  50.942
Эпоха: 06 | Время: 0m 57s
Перплексия (обучение):  13.824
Перплексия (валидация):  52.222
Эпоха: 07 | Время: 0m 58s
Перплексия (обучение):  13.439
Перплексия (валидация):  51.744
Эпоха: 08 | Время: 0m 57s
Перплексия (обучение):  13.178
Перплексия (валидация):  51.834
Эпоха: 09 | Время: 0m 58s
Перплексия (обучение):  12.868
Перплексия (валидация):  52.391
Эпоха: 10 | Время: 0m 57s
Перплексия (обучение):  12.620
Перплексия (валидация):  52.709
Эпоха: 11 | Время: 0m 57s
Перплексия (обучение):  12.336
Перплексия (валидация):  52.301
Эпоха: 12 | Время: 0m

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('translate_model.pt', map_location=device))

RuntimeError: ignored

In [ ]:
test_iter = Multi30k(split='test', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
test_dataset = TranslationDataset( test_iter, text_transform, PAD_IDX, ln_pair=ln_pair)
test_iterator = DataLoader(test_dataset, batch_size=128, collate_fn = test_dataset.pad_collate_fn)

#model.load_state_dict(torch.load('conala_model_attention_test.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Перплексия (валидация): {math.exp(test_loss):7.3f}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:181: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


Перплексия (валидация):  54.251


## Translate

In [ ]:
from typing import Sequence
from torchtext.vocab import Vocab
from torchtext.data.metrics import bleu_score

def predict( model, src, beam_size=5, max_len=20, device=None):
  """
  src - src_len x batch_size

  return tensor max_len x batch_size x beam_size
  """
  model.eval()
  if device == None:
      device = 'cuda' if torch.cuda.is_available() else 'cpu'

  src = src.to(model.device)
  batch_size = src.shape[1]

  # src_len x batch_size x hidden_size
  encoder_outputs = model.encode(src) 

  # 1 x batch_size
  input_trg = src[0].unsqueeze(0) 
  mask = model.create_pad_mask(src, input_trg)

  # 1 x batch_size x vocab_size
  first_decode, att, h_0 = model.decode(encoder_outputs, input_trg, mask, ) 
  first_decode = first_decode.log_softmax(-1)

  # 1 x batch_size x beam_size
  scores, indeces = first_decode.topk(beam_size) 

  # decoder hidden_state. num_layers * D, batch_size*beam_size, hidden
  h_0 = h_0.unsqueeze(2).repeat(1,1,beam_size,1).reshape(h_0.shape[0], batch_size*beam_size, -1) 

  # src_len x batch_size*beam_size x hidden_size
  encoder_outputs = encoder_outputs.unsqueeze(2).repeat(1,1,beam_size,1).reshape(src.shape[0], batch_size*beam_size, -1)

  # src_len x batch_size*beam_size
  src = src.unsqueeze(2).repeat(1,1,beam_size).reshape(src.shape[0], batch_size*beam_size)

  for i in range(1, max_len):
    input_trg = indeces[-1].view(1, batch_size * beam_size )
    mask = model.create_pad_mask(src, input_trg)
    dec_out , att, h_0 = model.decode(encoder_outputs, input_trg, mask, h_0=h_0)

    # 1 x batch_size*beam_size x vocab_size
    new_score = dec_out.log_softmax(-1) 

    # 1 x batch_size*beam_size x beam_size
    new_score, new_idx = new_score.topk(beam_size)

    new_score[:, torch.where(indeces[i-1].flatten() == model.eos_idx)[0]] = 0
    scores = scores.reshape(1, batch_size*beam_size, 1) + new_score

    lens_hyp = ((indeces != model.eos_idx) & (indeces != model.pad_idx)).sum(0, keepdim=True).reshape(1, batch_size*beam_size, 1)
    scores = scores/(lens_hyp+1)**0.5

    ######scores, new_idx = scores.topk(beam_size) #1 x  batch_size*beam_size x beam_size
    flatten_scores = scores.reshape(1,batch_size, beam_size ** 2)

    # 1 x batch_size x beam_size. order_scores [0 .. beam_size**2 - 1]
    scores, order_scores = flatten_scores.topk(beam_size) 

    
    # calc beam indexes that got top beam_size scores. beam_ids [0 .. beam_size - 1]
    beam_ids = torch.div(order_scores, beam_size, rounding_mode='floor') 

    #select beams with top scores from indeces and concat new idx.
    #start of sentences could repeat.

    indeces = torch.cat([
                         torch.gather(indeces, -1, beam_ids.repeat(indeces.shape[0], 1, 1)),
                         torch.gather(new_idx.view(1, batch_size, -1),-1,order_scores)
    ])

    #select beams with top scores from decoder hidden_state
    h_0 = h_0.reshape(h_0.shape[0], batch_size, beam_size, -1)
    h_0 = torch.gather(h_0, 2, beam_ids.unsqueeze(3).repeat(h_0.shape[0], 1, 1, h_0.shape[3]))
    h_0 = h_0.reshape(h_0.shape[0], batch_size*beam_size, -1)


  return indeces, scores
    
def data_to_device (data, device):
    if isinstance(data, Sequence):
        data = (d.to(device) for d in data)
    else:
        data = data.to(device)
    return data

def predict_with_model(model, iterator, vocab: Vocab,  device = None):
    model.eval()
    device = model.device
    condidate_corpus = []
    ref_corpus = []

    for batch in iterator:
      with torch.no_grad():
          src, trg = data_to_device(batch, device)
          #pred_trg, scores = predict(model,src, beam_size=1, max_len=trg.shape[0], device=device)
          #pred_trg = pred_trg[:,:,0]

          pred_trg = model(src, trg, 0)
          pred_trg = pred_trg.argmax(-1)
          
          for i in range(src.shape[1]):
              candidat = pred_trg[:,i][(pred_trg[:,i] != model.eos_idx) & (pred_trg[:,i] != model.pad_idx) ]
              candidat = vocab.lookup_tokens(list(candidat.cpu().numpy()))

              ref = trg[:,i][(trg[:,i] != model.eos_idx) & (trg[:,i] != model.pad_idx) ]
              ref = vocab.lookup_tokens(list(ref.cpu().numpy()))

              condidate_corpus.append(candidat)
              ref_corpus.append([ref])
    return condidate_corpus, ref_corpus


In [ ]:
#model.forward_mode = 'next_word'
condidat_corpus, ref_corpus = predict_with_model(model, valid_iterator, vocab_transform['en'])

In [ ]:
bleu_score(condidat_corpus, ref_corpus)

0.10913214868767868

In [ ]:
def display_attention(candidate, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels([''] + ['<bos>'] + [t.lower() for t in token_transform['de'](candidate)] + ['<eos>'], 
                       rotation=45)
    ax.set_yticklabels([''] + translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()



In [ ]:
model.eval()
sent = 'ich habe ein hund'.lower()
outputs, scores, a = model.predict(text_transform['de'](sent), beam_size = 5, max_len = len(sent.split(' ')) + 2)

for i in range(outputs.shape[0]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(outputs[i].cpu().numpy()))
    print(' '.join(text))

a i is a dog
if a dog with a
i am a dog with
someone am a dog with
i am a dog in


In [ ]:
example_idx = 8

src, trg = train_dataset[example_idx]
src_text, = ' '.join(vocab_transform[SRC_LANGUAGE].lookup_tokens(list(src))), 
trg_text  = ' '.join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(trg)))

print(f'src = {src_text}')
print(f'trg = {trg_text}')

outputs, scores, a = model.predict(src, beam_size = 5, max_len = 20)
for i in range(outputs.shape[0]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(outputs[i].cpu().numpy()))
    print(' '.join(text))
    #display_attention(src_text, text, a[i])
    

src = <bos> eine frau mit einer großen geldbörse geht an einem tor vorbei <eos>
trg = <bos> a woman with a large purse is walking by a gate <eos>
women with large purse walking past a gate <eos> a gate
woman with large purse walking past a gate <eos> a gate
lady with large purse walking past a gate <eos> a gate
an woman with a large purse walks by a gate <eos>
a woman with a large purse walks by a gate <eos>


In [ ]:
example_idx = 98
src, trg = val_dataset[example_idx]
src_text, = ' '.join(vocab_transform[SRC_LANGUAGE].lookup_tokens(list(src))), 
trg_text  = ' '.join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(trg)))

print(f'src = {src_text}')
print(f'trg = {trg_text}')

outputs, scores, a = model.predict(src, beam_size = 5, max_len = 40)
for i in range(outputs.shape[0]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(outputs[i].cpu().numpy()))
    print(' '.join(text))
    #display_attention(src_text, text, a[i])

src = <bos> einige männer sitzen in einem boot in der nähe eines strandes der mit gestapelten holzstämmen und stöcken bedeckt ist <eos>
trg = <bos> some men are sitting on a boat near a beach covered in stacked logs and sticks <eos>
many men sit in a boat near a beach with coconuts covered and hay <eos> <eos> hay
a few men sit in a boat near a beach with wooden coconuts and covered in wood
many men sit in a boat near a beach with coconuts covered and covered in coconuts <eos>
few men sit in a boat near a beach with coconuts covered and covered in coconuts <eos>
some men sit in a boat near a beach with coconuts covered and covered in coconuts <eos>


In [ ]:
SRC, TRG = next(iter(test_iterator))

In [ ]:
example_idx = 100

src, trg = SRC[:,example_idx], TRG[:,example_idx]
src_text, = ' '.join(vocab_transform[SRC_LANGUAGE].lookup_tokens(list(src))), 
trg_text  = ' '.join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(trg)))

print(f'src = {src_text}')
print(f'trg = {trg_text}')

outputs, scores, a = model.predict(src, beam_size = 5, max_len = 20)
for i in range(outputs.shape[0]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(outputs[i].cpu().numpy()))
    print(' '.join(text))
    #display_attention(src_text, text, a[i])

src = <bos> eine glückliche frau bereitet in einem coffee shop eine erfrischung zu <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
trg = <bos> a happy woman is preparing a <unk> at a coffee shop <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
a happy woman is preparing a drink in a coffee shop <eos> a coffee shop
the beautiful woman is preparing a drink in a coffee shop <eos> a coffee shop
the beautiful woman is preparing a drink at a coffee shop <eos> a coffee shop
one woman is preparing a drink in a coffee store <eos> a coffee shop <eos>
the beautiful woman is preparing a drink at a coffee shop <eos> <eos> <eos> <eos>


In [ ]:
idx, scores = predict(model,SRC , beam_size=1, max_len=20, device=device)

for i in range(idx.shape[2]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(idx[:,example_idx,i].cpu().numpy()))
    print(' '.join(text))
    #display_attention(src_text, text, a[i])



a happy woman is preparing a drink in a coffee shop <eos> a coffee shop <eos> <eos> <eos> <eos> <eos>


In [ ]:
#src.unsqueeze(1)
idx, scores = predict(model,src.unsqueeze(1) , beam_size=5, max_len=20, device=device)

for i in range(idx.shape[2]):
    text = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(idx[:,0,i].cpu().numpy()))
    print(' '.join(text))
    #display_attention(src_text, text, a[i])

a happy woman preparing a drink at a coffee shop <eos> a coffee shop <eos> <eos> <eos> <eos> <eos> <eos>
a happy woman preparing a drink at a coffee shop <eos> a coffee store <eos> <eos> <eos> <eos> <eos> <eos>
a happy woman is preparing a drink at a coffee shop <eos> a coffee shop <eos> <eos> <eos> <eos> <eos>
a happy woman is preparing a drink in a coffee shop <eos> a coffee shop <eos> <eos> <eos> <eos> <eos>
a happy woman is preparing a drink in a coffee store <eos> a coffee shop <eos> <eos> <eos> <eos> <eos>
